Rough Draft- Machine Learning Project
How does openness to trade affect economic growth in Africa ? 
Halleluiah Girum
March 5, 2023

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import statsmodels.api as sm
import openpyxl
from statsmodels.iolib.summary2 import summary_col

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA



from sklearn.preprocessing import StandardScaler


In [ ]:
## download datasets 
input = pd.read_csv(r'C:\Users\Halle\Documents\Econ1680\Project_1\data\inputs.csv',encoding='Latin-1')
output =  pd.read_csv(r'C:\Users\Halle\Documents\Econ1680\Project_1\data\output.csv')
polity = pd.read_csv(r'C:\Users\Halle\Documents\Econ1680\Project_1\data\polity.csv',encoding='Latin-1')
tarrifs = pd.read_excel(r'C:\Users\Halle\Documents\Econ1680\Project_1\data\tarrifs.xlsx')


In [ ]:
### Output summary 
output['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'] = output['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'].replace('..',np.nan)
output['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'] = output['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'].replace('..',np.nan)
output.drop(index=[1178,1179,1366],inplace=True)
output.drop(output[['Unnamed: 6', 'Unnamed: 7','Unnamed: 8', 'Unnamed: 9','Unnamed: 10','Unnamed: 11']],axis=1,inplace=True)


output['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'] = pd.to_numeric( output['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'] )
output['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'] = pd.to_numeric( output['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'] )
## creating a list of countries 
countries =[]
countries = output['Country Name']
countries.drop_duplicates(inplace=True)
countriesdf = pd.DataFrame(countries)


#creating a table with the mean GDP for each country 

meandf_country = output.groupby('Country Name')['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]','GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'].mean()
meandf_country.dropna(subset='GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]',inplace=True)
     
## create a table with the mean per each year 
meandf_year = output.groupby('Time')['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]','GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'].mean()
meandf_year.dropna(subset='GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]',inplace=True)


In [ ]:
 ## plot a graph of the change in GDP per country over time 
x= output['Time']
x.drop_duplicates(inplace=True)
x.drop(index=[0,2979,2982,2983],inplace=True)
x.astype(int)


y1= meandf_year['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]']
y2 = meandf_year['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]']
plt.plot(x,y1)
plt.plot(x,y2)
plt.legend(labels = ['GDP growth','GDP per capita growth'])
plt.xlabel('Time')
plt.ylabel('GDP growth (annual %)')
plt.title('Change in GDP')
plt.xticks(['1961','1970','1980','1990','2000','2010','2020'])
plt.show()



In [ ]:
## plot a graph of the mean change of GDP
output['Country Name'] = output['Country Name'].astype('|S') 
x= output['Country Name']
x.drop_duplicates(inplace=True)
x.drop(index=[2979],inplace=True)

y= meandf_country['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]']


plt.bar(x,y)
plt.title("Average GDP growth per country")
plt.xlabel("Country Name")
plt.ylabel("GDP growth per country")
plt.xticks(rotation=90)
plt.show()



In [ ]:
## plot a graph of the mean change of GDP per capita
output['Country Name'] = output['Country Name'].astype('|S') 
x= output['Country Name']
x.drop_duplicates(inplace=True)
x.drop(index=[2979],inplace=True)

y= meandf_country['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]']


plt.bar(x,y)
plt.title("Average GDP per capita growth vs Country")
plt.xlabel("Country Name")
plt.ylabel("GDP per capita growth")
plt.xticks(rotation=90)
plt.show()


In [ ]:
## mean and standard deviation of output 

output['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]'].describe()

output['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'].describe()

In [ ]:
## summary statistics for trade openness 
## create a new df with imports and exports 
tradeDf = input[['Country Name','Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]','Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]','Time']]
tradeDf['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'] = tradeDf['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'].replace('..',np.nan)
tradeDf.dropna(subset=['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]','Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'],inplace=True)
tradeDf['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'] = pd.to_numeric( tradeDf['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'] )
tradeDf['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'] = pd.to_numeric( tradeDf['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'] )

## get the mean of exports and imports by country 
meandf_country2 = tradeDf.groupby('Country Name')['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]','Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'].mean()
## get the mean of exports and imports by year 
meandf_year2 = tradeDf.groupby('Time')['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]','Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'].mean()

In [ ]:
### plotting graph of change in imports and exports 
x= tradeDf['Time']
x.drop_duplicates(inplace=True)
x =x.sort_values()
y1= meandf_year2['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]']
y2 = meandf_year2['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]']
plt.plot(x,y1)
plt.plot(x,y2)
plt.legend(labels = ['Exports','Imports'])
plt.xlabel('Time')
plt.xticks(['1960','1970','1980','1990','2000','2010','2020'])
plt.ylabel('Exports/Imports % of GDP')
plt.title('Exports/Imports vs Time')
plt.show()


In [ ]:
## plotting means of each country,  exports

x= tradeDf['Country Name']
x.drop_duplicates(inplace=True)
y= meandf_country2['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]']
plt.bar(x,y)
plt.title("Exports vs Countries")
plt.xlabel("Country Name")
plt.ylabel("Exports % of GDP")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# mean imports of countries plot
x= tradeDf['Country Name']
x.drop_duplicates(inplace=True)
y= meandf_country2['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]']
plt.bar(x,y)
plt.title("Imports vs Countries")
plt.xlabel("Country Name")
plt.ylabel("Imports % of GDP")
plt.xticks(rotation=90)
plt.show()

In [ ]:
## trade summary statisics 

tradeDf['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'].describe()
tradeDf['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]'].describe()

In [ ]:
## change variable names 
tradeDf.rename(columns={'Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]' : 'Exports', 'Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]' : 'Imports'},inplace=True)

## Principal Component Analysis 
## first normalize percentages between 0 &1, then standardize
scaler = StandardScaler()
tradeScaled = scaler.fit_transform(tradeDf[['Exports','Imports']])


tradeScaled = pd.DataFrame(tradeScaled)
tradeScaled.rename(columns ={0:'Exports',1:'Imports'},inplace=True)
## pca
pca = PCA(n_components=1)
pc1 =pca.fit_transform(tradeScaled)
## adding the first and second PCA as columns to df_scaled 
tradeScaled['pc1'] = pc1[:,0]





In [ ]:

tarrifsdf = pd.DataFrame(tarrifs)
tarrifsdf = tarrifsdf.rename(columns={'Unnamed: 0' : 'Country Name', 'Unnamed: 1' : 'Time', 'Unnamed: 2' : 'Simple average duty', 'Unnamed: 3' : 'Maximum duty',
                                    'Unnamed: 4' : 'Duty free', 'Unnamed: 5' : 'Imports duty free', 'Unnamed: 6' : 'Number of distinct duty rates', 
                                    'Unnamed: 7' : 'Number of applied tariff lines', 'Unnamed: 8' : 'Number of applied tariff lines', 'Unnamed: 9' : 'Binding coverage',
                                    'Unnamed: 10' : 'Tariffs greater than 3* national average', 'Unnamed: 11' : 'Tariffs greater than 15%'
    
})
tarrifsdf.drop(0,inplace=True)

## add more tarrif varaibles from input dataset by merging 

tarrifsdf['Time'] = pd.to_numeric(tarrifsdf['Time'])
tarrifs_ = input[['Time','Country Name','Taxes on exports (% of tax revenue) [GC.TAX.EXPT.ZS]','Taxes on international trade (% of revenue) [GC.TAX.INTT.RV.ZS]','Customs and other import duties (% of tax revenue) [GC.TAX.IMPT.ZS]','Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]',
                 'Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]']]
tarrifsdf = pd.merge(tarrifsdf,tarrifs_,on=['Country Name','Time'])

In [ ]:
## Principal component analysis with tarrifs 
## scale data
tarrifsdf = tarrifsdf.replace('..',np.nan)
scaler = StandardScaler()
tarrifscaled = scaler.fit_transform(tarrifsdf[['Simple average duty','Maximum duty','Duty free','Imports duty free','Number of distinct duty rates','Number of applied tariff lines',
                                               'Binding coverage','Tariffs greater than 3* national average','Tariffs greater than 15%',
                                               'Taxes on exports (% of tax revenue) [GC.TAX.EXPT.ZS]','Taxes on international trade (% of revenue) [GC.TAX.INTT.RV.ZS]','Customs and other import duties (% of tax revenue) [GC.TAX.IMPT.ZS]','Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]',
                 'Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]']])

tarrifscaleddf = pd.DataFrame(tarrifscaled)

## first standardize the data 
# mergedTradeScaled = scaler.fit_transform(mergedTrade[['Simple average duty','Maximum duty','Duty free','Imports duty free','Number of distinct duty rates','Number of applied tariff lines',
#                                                'Binding coverage','Tariffs greater than 3* national average','Tariffs greater than 15%','Exports','Imports']])
# mergedTrade = pd.DataFrame(mergedTrade)
# countries = mergedTrade['Country Name']
# mergedTradeScaled = pd.DataFrame(mergedTradeScaled)
# mergedTradeScaled['Country Name'] = countries
# mergedTradeScaled['Time'] = mergedTrade['Time']


In [ ]:

## first drop variables that are all empty 

tarrifscaleddf.drop(columns=[6,7,8,9],inplace=True)

## impute to replace missing values with the mean of that column 
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
df_imputed = imp_mean.fit_transform(tarrifscaleddf)

## use PCA to combine variation in all of the columns in the tarrifscaleddf
pca = PCA(n_components=1)
pc1 =pca.fit_transform(df_imputed)
## adding the first and second PCA as columns to df_scaled 
tarrifscaleddf['pc1'] = pc1[:,0]


In [ ]:
## trade openness index 1 

## add and drop appropriate columns to merge with other inputs 
# tarrifscaleddf.drop(columns=[0,1,2,3,4,5,10,11,12,13,14],inplace=True)
tarrifscaleddf['Country Name'] = tarrifsdf['Country Name']
tarrifscaleddf['Time'] = tarrifsdf['Time']
## merge with other inputs 
inputs2 = pd.merge(input,tarrifscaleddf,on=['Country Name','Time'])
inputs2.rename(columns={'pc1' : 'Trade openness'},inplace=True)
## create dataframe with both inputs and output 
output.dropna(subset=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'],inplace=True)
output['Time'] = output['Time'].astype(float)
df2 = pd.merge(inputs2,output,on=['Country Code','Time'])


In [ ]:
## creating country fixed effects for trade openness index 1 

df_dummies2 = pd.get_dummies(df2,columns=['Country Name_x','Time'],drop_first=True)
df_dummies2.rename(columns={'Country Name_x':'Country Name'},inplace=True)
df_dummies2 = df_dummies2.replace('..',np.nan)
## splitting the data into training and testing sets
y1=df_dummies2['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]']
y2 = df_dummies2['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]']
X=df_dummies2.drop(columns=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]','Time Code_x','Time Code_y','Country Code','Country Name_y'])
## use imputer to replace missing values with the mean of the column 
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
X_transform =imp_mean.fit_transform(X)
imp_constant = SimpleImputer(missing_values=np.nan,strategy='constant')
y1 =pd.DataFrame(y1)
y2 = pd.DataFrame(y2)
y1_transform = imp_mean.fit_transform(y1)
y2_transform = imp_mean.fit_transform(y2)

scaler = StandardScaler()
X_transform  = scaler.fit_transform(X_transform)
y1_transform = scaler.fit_transform(y1_transform)
## split into training and testing data sets 
X_train, X_test, y_train, y_test = train_test_split(X_transform, y1_transform, test_size=0.1, random_state=0)


In [ ]:
## trade openness index 2 

## add name and time columns
tradeScaled['Country Name'] = tradeDf['Country Name']
tradeScaled['Time'] = tradeDf['Time']
tradeScaled.drop(columns=['Exports','Imports'])
#merge with other input variables 
tradeScaled['Time'] = tradeScaled['Time'].astype(float)
inputs3 = pd.merge(input,tradeScaled,on=['Country Name','Time'])
inputs3.rename(columns={'pc1' : 'Trade openness'},inplace=True)
## create dataframe with both inputs and output 
output.dropna(subset=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'],inplace=True)
output['Time'] = output['Time'].astype(float)
df3 = pd.merge(inputs2,output,on=['Country Code','Time'])

In [ ]:
## creating country fixed effects for trade openness index 2

df_dummies3 = pd.get_dummies(df3,columns=['Country Name_x','Time'],drop_first=True)
df_dummies3.rename(columns={'Country Name_x':'Country Name'},inplace=True)
df_dummies3 = df_dummies3.replace('..',np.nan)

## splitting the data into training and testing sets
y1=df_dummies3['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]']
y2 = df_dummies3['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]']
X=df_dummies3.drop(columns=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]','Time Code_x','Time Code_y','Country Code','Country Name_y'])

## use imputer to replace missing values with the mean of the column 
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
X_transform =imp_mean.fit_transform(X)
imp_constant = SimpleImputer(missing_values=np.nan,strategy='constant')
y1 =pd.DataFrame(y1)
y2 = pd.DataFrame(y2)
y1_transform = imp_mean.fit_transform(y1)
y2_transform = imp_mean.fit_transform(y2)

scaler = StandardScaler()
X_transform  = scaler.fit_transform(X_transform)
y1_transform = scaler.fit_transform(y1_transform)
## split into training and testing data sets 
X_train, X_test, y_train, y_test = train_test_split(X_transform, y1_transform, test_size=0.1, random_state=0)

In [ ]:
## traditional trade openness measure - trade openness index 3 
tradeopen = pd.read_csv(r'C:\Users\Halle\Documents\Econ1680\Project_1\data\trade-openness.csv')

codes = input['Country Code']
codes.drop_duplicates(inplace=True)
# codes.drop(index=[2976,2981,2989,2993,2994,2995,3004,3005],inplace=True)

tradeopen.rename(columns={'Code' : 'Country Code','Year':'Time'},inplace=True)
inputs = pd.merge(tradeopen,codes,how='right')

input.dropna(subset=['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]'],inplace=True)
input['Time'] = input['Time'].astype(float)

## adding trade openness measure to orginal input dataframe 
X = pd.merge(input,inputs,on=['Country Code','Time'])

## cleaning up data, getting rid of other trade variables 
X.drop(X[['Entity','Unnamed: 4','Unnamed: 5','Unnamed: 6']],axis=1,inplace=True)
# X.drop(X[['Customs and other import duties (% of tax revenue) [GC.TAX.IMPT.ZS]']],axis=1,inplace=True)
# X.drop(X[['Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]']],axis=1,inplace=True)
# X.drop(X[['Exports of goods and services (annual % growth) [NE.EXP.GNFS.KD.ZG]']],axis=1,inplace=True)
# X.drop(X[['Fuel exports (% of merchandise exports) [TX.VAL.FUEL.ZS.UN]']],axis=1,inplace=True)
# X.drop(X[['Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]']],axis=1,inplace=True)
# X.drop(X[['Imports of goods and services (annual % growth) [NE.IMP.GNFS.KD.ZG]']],axis=1,inplace=True)
# X.drop(X[['Taxes on exports (% of tax revenue) [GC.TAX.EXPT.ZS]']],axis=1,inplace=True)
# X.drop(X[['Taxes on international trade (% of revenue) [GC.TAX.INTT.RV.ZS]']],axis=1,inplace=True)
# X.drop(X[['Share of tariff lines with specific rates, all products (%) [TM.TAX.MRCH.SR.ZS]']],axis=1,inplace=True)
# X.drop(X[['Share of tariff lines with specific rates, manufactured products (%) [TM.TAX.MANF.SR.ZS]']],axis=1,inplace=True)
# X.drop(X[['Share of tariff lines with specific rates, primary products (%) [TM.TAX.TCOM.SR.ZS]']],axis=1,inplace=True)


In [ ]:
## merge input and output by time and country code
output.dropna(subset=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]'],inplace=True)
output['Time'] = output['Time'].astype(float)
df = pd.merge(X,output,on=['Country Code','Time'])



In [ ]:
## creating country fixed effects and time fixed effects 

df_dummies = pd.get_dummies(df,columns=['Country Name_x','Time'],drop_first=True)
df_dummies.rename(columns={'Country Name_x':'Country Name'},inplace=True)

In [ ]:
## split into training and testing data sets 
y1=df_dummies['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]']
y2 = df_dummies['GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]']
X=df_dummies.drop(columns=['GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]','GDP per capita growth (annual %) [NY.GDP.PCAP.KD.ZG]','Time Code_x','Time Code_y','Time Code','Country Code','Country Name_y'])

In [ ]:
## use imputer to replace missing values with the mean of the column 
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
X_transform =imp_mean.fit_transform(X)
imp_constant = SimpleImputer(missing_values=np.nan,strategy='constant')
y1 =pd.DataFrame(y1)
y2 = pd.DataFrame(y2)
y1_transform = imp_mean.fit_transform(y1)
y2_transform = imp_mean.fit_transform(y2)

scaler = StandardScaler()
X_transform  = scaler.fit_transform(X_transform)
y1_transform = scaler.fit_transform(y1_transform)
## split into training and testing data sets 
X_train, X_test, y_train, y_test = train_test_split(X_transform, y1_transform, test_size=0.1, random_state=0)

In [ ]:
## run a lasso regression to select features 
# Define model
lassoReg = make_pipeline(StandardScaler(with_mean=False), Lasso())

# Define parameter grid to search over using grid search
alphas = [0.000001 * np.sqrt(X.shape[0]), 0.0001 * np.sqrt(X.shape[0]), 0.01 * np.sqrt(X.shape[0]), 1 * np.sqrt(X.shape[0])]
params = {'lasso__alpha' : alphas}

# Set up the grid search
gsLasso = GridSearchCV(lassoReg, params, n_jobs=-1, cv=10)

# Fit gs to data
gsLasso.fit(X_transform, y1_transform)

# Check best alpha
best_alpha =list(gsLasso.best_params_.values())[0] / np.sqrt(X_train.shape[0])
print(best_alpha)

In [ ]:
# Run lasso regression
lassoReg = make_pipeline(StandardScaler(with_mean=False), Lasso(alpha=0.0105 * np.sqrt(X_train.shape[0])))
lassoReg.fit(X_train, y_train)

# Add coefficients to dataframe
df_results = pd.DataFrame({'Variable':X.columns})
df_results['Coeff LASSO'] = lassoReg['lasso'].coef_
print(df_results)
